## **Telco Customer Churn Prediction Model**

In [1]:
# Import Data Manipulation Libraries
import numpy as np
import pandas as pd
# Import Data Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
# Import Machine Learning Libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
# Import Imbalanced Data Handling Libraries
from imblearn.over_sampling import SMOTE
# Suppress Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Data Loading : Data Ingestion
def data_ingestion(file_path):
    try:
        data = pd.read_csv(file_path)
        print("Data Ingestion Successful!")
        return data
    except Exception as e:
        print(f"Error during data ingestion: {e}")
        return None

In [7]:
def main():
    # Load the dataset
    file_path = r'C:\MachineLearningModel_Deployment\data\raw\churn.csv'  # Update this path
    data = data_ingestion(file_path)
    
    if data is not None:
        print(data.head())
        print(data.info())
        print(data.describe())
        print(data.columns)
        # Further processing and model training code goes here
        
main()

Data Ingestion Successful!
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV Streaming